In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
import re
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('indonesian')

In [2]:
ds = pd.read_csv(r"D:\UI\ANMEDSOS\Praktikum\KA\irrelevant.csv")

In [3]:
# Sebelum Menambahkan 
ds

,label,id,waktu,text
0,IRRELEVANT,1.390000e+18,Fri Apr 23 21:22:48 +0000 2021,Masa mimpi telat masuk kelas pas kuliah offlin...
1,IRRELEVANT,1.390000e+18,Fri Apr 23 11:13:44 +0000 2021,@sbmptnfess 1. Kelas 12 semester 1\n2. Semenja...
2,IRRELEVANT,1.390000e+18,Thu Apr 22 15:10:14 +0000 2021,#NAME?
3,IRRELEVANT,1.390000e+18,Thu Apr 22 15:03:48 +0000 2021,@treasuremenfess @treasuremembers Gue udah ada...
4,IRRELEVANT,1.390000e+18,Thu Apr 22 13:30:00 +0000 2021,"@schfess dulu waktu sekolah offline, kelasku k..."
...,...,...,...,...
10058,IRRELEVANT,1.410000e+18,Mon Jun 28 04:21:51 +0000 2021,hari ini kelas gak ada yg tatap muka sama aja ...
10059,IRRELEVANT,1.410000e+18,Mon Jun 28 02:48:54 +0000 2021,@schfess Maaf tapi sklh gue udh ada rencana bu...
10060,IRRELEVANT,1.410000e+18,Mon Jun 28 01:46:02 +0000 2021,"@lilaccountz Walas tatap muka : ngajar matpel,..."
10061,IRRELEVANT,1.410000e+18,Sun Jun 27 07:45:00 +0000 2021,RT @waIIahu: fyi aja nich buat adek adek yang ...


In [4]:
ds['text'] = ds['text'].str.lower()
ds.dropna(subset=['label'], inplace=True)
ds

,label,id,waktu,text
0,IRRELEVANT,1.390000e+18,Fri Apr 23 21:22:48 +0000 2021,masa mimpi telat masuk kelas pas kuliah offlin...
1,IRRELEVANT,1.390000e+18,Fri Apr 23 11:13:44 +0000 2021,@sbmptnfess 1. kelas 12 semester 1\n2. semenja...
2,IRRELEVANT,1.390000e+18,Thu Apr 22 15:10:14 +0000 2021,#name?
3,IRRELEVANT,1.390000e+18,Thu Apr 22 15:03:48 +0000 2021,@treasuremenfess @treasuremembers gue udah ada...
4,IRRELEVANT,1.390000e+18,Thu Apr 22 13:30:00 +0000 2021,"@schfess dulu waktu sekolah offline, kelasku k..."
...,...,...,...,...
10058,IRRELEVANT,1.410000e+18,Mon Jun 28 04:21:51 +0000 2021,hari ini kelas gak ada yg tatap muka sama aja ...
10059,IRRELEVANT,1.410000e+18,Mon Jun 28 02:48:54 +0000 2021,@schfess maaf tapi sklh gue udh ada rencana bu...
10060,IRRELEVANT,1.410000e+18,Mon Jun 28 01:46:02 +0000 2021,"@lilaccountz walas tatap muka : ngajar matpel,..."
10061,IRRELEVANT,1.410000e+18,Sun Jun 27 07:45:00 +0000 2021,rt @waiiahu: fyi aja nich buat adek adek yang ...


In [5]:
import time
from datetime import timedelta
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

start_time = time.time()

In [6]:
ds['text'] = ds.text.astype(str)
swfactory = StopWordRemoverFactory()
stopword = swfactory.create_stop_word_remover()
stfactory = StemmerFactory()
stemmer = stfactory.create_stemmer()
tok = WordPunctTokenizer()

removehtml = r'http\S+'
removemention = r'@\S+'
removewww = r'www.\S+'
removepic = r'pic\S+'

In [7]:
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    soup1 = BeautifulSoup(souped, 'html.parser')
    souped1 = soup1.get_text()
    stripped = re.sub(removehtml,'',souped1)
    stripped = re.sub(removemention,'',stripped)
    stripped = re.sub(removewww, '', stripped)
    stripped = re.sub(removepic, '', stripped)
    letters_only = re.sub('[^a-zA-Z]',' ',stripped)
    no_elongated = re.sub(r'(.)\1+', r'\1\1', letters_only)
    stopw = stopword.remove(no_elongated)
    stemmed = stemmer.stem(stopw)
    words = [x for x in tok.tokenize(stemmed) if len(x) > 2]
    return (" ".join(words)).strip()

In [10]:
print("Cleaning and parsing the comments...\n")
clean_tweet_texts = []
for i in range(0,10063):
    if((i+1)%100 == 0):
        print("Comments %d of %d has been processed" % (i+1, 100))
    clean_tweet_texts.append(tweet_cleaner(ds['text'][i]))
    
clean_df = pd.DataFrame(clean_tweet_texts, columns=['text'])
clean_df['waktu'] = ds.waktu
clean_df['jumlah kata'] = clean_df.text.apply(lambda x: len(x.split()))

clean_df['waktu'] =  pd.to_datetime(clean_df['waktu'], format='%a %b %d %H:%M:%S %z %Y')
clean_df['jam'] = clean_df.waktu.dt.hour
clean_df['jam'] = pd.cut(clean_df.jam, bins=[0,8,16,24], labels=['pagi','siang','malam'])


clean_df['label'] = ds.label

Cleaning and parsing the comments...

Comments 100 of 100 has been processed
Comments 200 of 100 has been processed
Comments 300 of 100 has been processed
Comments 400 of 100 has been processed
Comments 500 of 100 has been processed
Comments 600 of 100 has been processed
Comments 700 of 100 has been processed
Comments 800 of 100 has been processed
Comments 900 of 100 has been processed
Comments 1000 of 100 has been processed
Comments 1100 of 100 has been processed
Comments 1200 of 100 has been processed
Comments 1300 of 100 has been processed
Comments 1400 of 100 has been processed
Comments 1500 of 100 has been processed
Comments 1600 of 100 has been processed
Comments 1700 of 100 has been processed
Comments 1800 of 100 has been processed
Comments 1900 of 100 has been processed
Comments 2000 of 100 has been processed
Comments 2100 of 100 has been processed
Comments 2200 of 100 has been processed
Comments 2300 of 100 has been processed
Comments 2400 of 100 has been processed
Comments 25

In [11]:
clean_df

,text,waktu,jumlah kata,jam,label
0,masa mimpi telat masuk kelas pas kuliah offline,2021-04-23 21:22:48+00:00,8,malam,IRRELEVANT
1,kelas semester semenjak ikut bimbel offline it...,2021-04-23 11:13:44+00:00,19,siang,IRRELEVANT
2,name,2021-04-22 15:10:14+00:00,1,siang,IRRELEVANT
3,gue udah matkul offline kok nder walaupun cuma...,2021-04-22 15:03:48+00:00,35,siang,IRRELEVANT
4,dulu waktu sekolah offline kelas kan pake roll...,2021-04-22 13:30:00+00:00,34,siang,IRRELEVANT
...,...,...,...,...,...
10058,hari kelas gak tatap muka sama aja libur wkwk ...,2021-06-28 04:21:51+00:00,19,pagi,IRRELEVANT
10059,maaf sklh gue udh rencana buat full offline ba...,2021-06-28 02:48:54+00:00,27,pagi,IRRELEVANT
10060,walas tatap muka ngajar matpel mantau anak kel...,2021-06-28 01:46:02+00:00,37,pagi,IRRELEVANT
10061,fyi aja nich buat adek adek gatau jadi kuliah ...,2021-06-27 07:45:00+00:00,19,pagi,IRRELEVANT


In [12]:
#clean_df.fillna(0)
clean_df['jam'].fillna('pagi', inplace =True)
clean_df['label'].fillna('0', inplace =True)

In [13]:
clean_duplicate = clean_df.drop_duplicates(subset='text', keep="last")

In [14]:
clean_duplicate

,text,waktu,jumlah kata,jam,label
1,kelas semester semenjak ikut bimbel offline it...,2021-04-23 11:13:44+00:00,19,siang,IRRELEVANT
3,gue udah matkul offline kok nder walaupun cuma...,2021-04-22 15:03:48+00:00,35,siang,IRRELEVANT
5,waktu offline dlu ujan deres bgt pramuka cance...,2021-04-22 12:16:40+00:00,39,siang,IRRELEVANT
6,jadi pas kbm offline pas pulsek aku sering sen...,2021-04-22 11:45:06+00:00,42,siang,IRRELEVANT
7,mimpi masuk itb offline duduk kelas ftsl pake ...,2021-04-22 09:04:37+00:00,14,siang,IRRELEVANT
...,...,...,...,...,...
10058,hari kelas gak tatap muka sama aja libur wkwk ...,2021-06-28 04:21:51+00:00,19,pagi,IRRELEVANT
10059,maaf sklh gue udh rencana buat full offline ba...,2021-06-28 02:48:54+00:00,27,pagi,IRRELEVANT
10060,walas tatap muka ngajar matpel mantau anak kel...,2021-06-28 01:46:02+00:00,37,pagi,IRRELEVANT
10061,fyi aja nich buat adek adek gatau jadi kuliah ...,2021-06-27 07:45:00+00:00,19,pagi,IRRELEVANT


In [15]:
clean_duplicate.to_csv('KA\data\Irrelevant Clean No Duplicate.csv',encoding='utf-8')